# Income & Benefit - Data Pull

## Import Needed Libraries

In [13]:
import pandas as pd
import requests
import textwrap
from time import sleep
import json
import xlsxwriter
import os
import openpyxl
from openpyxl import workbook, load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import numpy as np

## Setting 'Key'

In [14]:
key = 'PLEASE INSERT YOUR KEY HERE'

## Variables for API Calls

Assigning all the variables for each call to a python variable to be inserterted in the 
API call as a string.

In [15]:
total_households = ('NAME,DP03_0051E')
	
th_range = ('NAME,DP03_0052E,DP03_0052PE,DP03_0053E,DP03_0053PE,DP03_0054E,'
            'DP03_0054PE,DP03_0055E,DP03_0055PE,DP03_0056E,DP03_0056PE,'
            'DP03_0057E,DP03_0057PE,DP03_0058E,DP03_0058PE,DP03_0059E,'
            'DP03_0059PE,DP03_0060E,DP03_0060PE,DP03_0061E,DP03_0061PE'
            )
	
th_w_earn = ('NAME,DP03_0064E')
	
th_w_ss = ('NAME,DP03_0066E,DP03_0066PE')
	
th_w_ri = ('NAME,DP03_0068E,DP03_0068PE')
	
th_w_ssi = ('NAME,DP03_0070E,DP03_0070PE')
	
th_w_cpa = ('NAME,DP03_0072E,DP03_0072PE')
	
th_w_snap = ('NAME,DP03_0074E,DP03_0074PE')
	
families = ('NAME,DP03_0075E')
	
fm_range = ('NAME,DP03_0076E,DP03_0076PE,DP03_0077E,DP03_0077PE,DP03_0078E,'
            'DP03_0078PE,DP03_0079E,DP03_0079PE,DP03_0080E,DP03_0080PE,'
            'DP03_0081E,DP03_0081PE,DP03_0082E,DP03_0082PE,DP03_0083E,'
            'DP03_0083PE,DP03_0084E,DP03_0084PE,DP03_0085E,DP03_0085PE'
            )
	
per_capita = ('NAME,DP03_0088E')
	
non_family = ('NAME,DP03_0089E')
	
worker_earn = ('NAME,DP03_0092E')
	
worker_ft_earn_male = ('NAME,DP03_0093E')
	
worker_ft_earn_female = ('NAME,DP03_0094E')



In [16]:
get_vars = [total_households,th_range,th_w_earn,th_w_ss,th_w_ri,th_w_ssi,
            th_w_cpa,th_w_snap,families,fm_range,per_capita,non_family,
            worker_earn,worker_ft_earn_male,worker_ft_earn_female
            ]

df_vars = ['DF_total_households','DF_th_range','DF_th_w_earn','DF_th_w_ss',
           'DF_th_w_ri','DF_th_w_ssi','DF_th_w_cpa','DF_th_w_snap',
           'DF_families','DF_fm_range','DF_per_capita','DF_non_family',
           'DF_worker_earn','DF_worker_ft_earn_male','DF_worker_ft_earn_female'
           ]

### API Calls

completing 15 calls to the API and dumping each into a named dataframe in a dict

In [17]:
# API call parameters
url = 'https://api.census.gov/data/2020/acs/acs5/profile'
how = 'state:*'
where = 'county:*'

# Create an empty dictionary to store the data frames
dfs = {}

# Iterate through get_vars and df_vars simultaneously
for var, name in zip(get_vars, df_vars):
    r = requests.request('GET', url, params={"get": var, "for": where, "in": how, "key": key})

    # Check if the API call was successful
    if r.status_code == 200:
        data = r.json()

        # Create the data frame using the returned JSON data
        df_data = data[1:]  # Skip the first row which contains the column names
        dfs[name] = pd.DataFrame(df_data, columns=data[0])

        # Join 'state' and 'county' columns into a new column 'FIPS'
        dfs[name]['FIPS'] = dfs[name]['state'] + dfs[name]['county']

        # Convert specific columns to desired data types
        columns_to_convert = ['NAME', 'state', 'county', 'FIPS']
        columns_to_convert = [col for col in columns_to_convert if col in dfs[name].columns]
        columns_to_convert_int = dfs[name].columns.difference(columns_to_convert)
        columns_to_convert_float = []

        # Handle integer and float conversion separately
        for col in columns_to_convert_int:
            try:
                dfs[name][col] = dfs[name][col].astype(int)
            except ValueError:
                # If it cannot be converted to int, add it to the float conversion list
                columns_to_convert_float.append(col)

        for col in columns_to_convert_float:
            dfs[name][col] = dfs[name][col].astype(float)

        # Display the created data frame
        print(f"Data frame '{name}':")
        print(dfs[name])
    else:
        # API call failed
        print(f"API call for '{var}' failed. Status code:", r.status_code)

Data frame 'DF_total_households':
                             NAME  DP03_0051E state county   FIPS
0         Autauga County, Alabama       21559    01    001  01001
1         Baldwin County, Alabama       84047    01    003  01003
2         Barbour County, Alabama        9322    01    005  01005
3            Bibb County, Alabama        7259    01    007  01007
4          Blount County, Alabama       21205    01    009  01009
...                           ...         ...   ...    ...    ...
3216   Renville County, Minnesota        6032    27    129  27129
3217     Roseau County, Minnesota        5940    27    135  27135
3218  Sherburne County, Minnesota       32791    27    141  27141
3219     Steele County, Minnesota       14836    27    147  27147
3220       Todd County, Minnesota        9888    27    153  27153

[3221 rows x 5 columns]
Data frame 'DF_th_range':
                             NAME  DP03_0052E  DP03_0052PE  DP03_0053E  \
0         Autauga County, Alabama        1331    

### Verify Pulls from API

I should have 15 Dataframes total.

- There should be 8 dataframes of:
    - Shape: 3221 rows, 5 columns
- There should be 5 dataframes of:
    - Shape: 3221 rows, 6 columns
- There should be 2 dataframes of:
    - Shape: 3221 rows, 24 columns

In [18]:
# Print the number of data frame names stored in dfs dictionary
print(f"Number of data frames: {len(dfs)}")

print("Data Frames in dfs:")
for name, df in dfs.items():
    # Get the shape of the data frame
    rows, cols = df.shape

    # Print the data frame name and shape
    print(f"Data frame '{name}':")
    print(f"Shape: {rows} rows, {cols} columns")

Number of data frames: 15
Data Frames in dfs:
Data frame 'DF_total_households':
Shape: 3221 rows, 5 columns
Data frame 'DF_th_range':
Shape: 3221 rows, 24 columns
Data frame 'DF_th_w_earn':
Shape: 3221 rows, 5 columns
Data frame 'DF_th_w_ss':
Shape: 3221 rows, 6 columns
Data frame 'DF_th_w_ri':
Shape: 3221 rows, 6 columns
Data frame 'DF_th_w_ssi':
Shape: 3221 rows, 6 columns
Data frame 'DF_th_w_cpa':
Shape: 3221 rows, 6 columns
Data frame 'DF_th_w_snap':
Shape: 3221 rows, 6 columns
Data frame 'DF_families':
Shape: 3221 rows, 5 columns
Data frame 'DF_fm_range':
Shape: 3221 rows, 24 columns
Data frame 'DF_per_capita':
Shape: 3221 rows, 5 columns
Data frame 'DF_non_family':
Shape: 3221 rows, 5 columns
Data frame 'DF_worker_earn':
Shape: 3221 rows, 5 columns
Data frame 'DF_worker_ft_earn_male':
Shape: 3221 rows, 5 columns
Data frame 'DF_worker_ft_earn_female':
Shape: 3221 rows, 5 columns


### Checking for data errors

I should find no missing data.

In [19]:
for df_name, df in dfs.items():
    # Find the rows with missing data
    rows_with_missing_data = df[df.isnull().any(axis=1)]

    # Find the columns with missing data
    columns_with_missing_data = df.columns[df.isnull().any()]

    # Check if there is missing data in the dataframe
    if not rows_with_missing_data.empty or len(columns_with_missing_data) > 0:
        print(f"Missing data found in DataFrame '{df_name}'")

        # Replace NaN values with string 'NaN'
        df = df.replace({np.nan: 'NaN'})

        # Print the rows with missing data
        print("Rows with missing data:")
        print(rows_with_missing_data)

        # Print the columns with missing data
        print("Columns with missing data:")
        print(columns_with_missing_data)
        print()

        # Search for 'NaN' string in the dataframe
        rows_with_nan_string = df[df.eq('NaN').any(axis=1)]
        columns_with_nan_string = df.columns[df.eq('NaN').any()]

        # Print the rows with 'NaN' string
        print("Rows with 'NaN' string:")
        print(rows_with_nan_string)

        # Print the columns with 'NaN' string
        print("Columns with 'NaN' string:")
        print(columns_with_nan_string)
        print()
    else:
        print(f"No missing data found in DataFrame '{df_name}'")

No missing data found in DataFrame 'DF_total_households'
No missing data found in DataFrame 'DF_th_range'
No missing data found in DataFrame 'DF_th_w_earn'
No missing data found in DataFrame 'DF_th_w_ss'
No missing data found in DataFrame 'DF_th_w_ri'
No missing data found in DataFrame 'DF_th_w_ssi'
No missing data found in DataFrame 'DF_th_w_cpa'
No missing data found in DataFrame 'DF_th_w_snap'
No missing data found in DataFrame 'DF_families'
No missing data found in DataFrame 'DF_fm_range'
No missing data found in DataFrame 'DF_per_capita'
No missing data found in DataFrame 'DF_non_family'
No missing data found in DataFrame 'DF_worker_earn'
No missing data found in DataFrame 'DF_worker_ft_earn_male'
No missing data found in DataFrame 'DF_worker_ft_earn_female'


### Modify Column Locations

dropping common columns, then adding back to the end of the index for each frame<br>
inplace and concat to ensure no data loss
>['NAME','state','county','FIPS']<br>



In [20]:
# Iterate over each DataFrame in the dictionary
for name, df in dfs.items():
    # Get the common columns from the DataFrame
    common_cols = df[['NAME', 'state', 'county', 'FIPS']]
    
    # Drop the common columns from the DataFrame
    df.drop(columns=['NAME', 'state', 'county', 'FIPS'], inplace=True)
    
    # Concatenate the DataFrame with the common columns at the end
    df = pd.concat([df, common_cols], axis=1)
    
    # Update the DataFrame in the dictionary
    dfs[name] = df

### Verify Column Locations

print the name of the dataframe and list the columns to verify operation success


In [21]:
# Iterate over each DataFrame in the dictionary
for name, df in dfs.items():
    # Print the name of the DataFrame
    print(f"DataFrame '{name}':")
    
    # Print the list of column names
    print(df.columns.tolist())
    print()  # Print an empty line for separation

DataFrame 'DF_total_households':
['DP03_0051E', 'NAME', 'state', 'county', 'FIPS']

DataFrame 'DF_th_range':
['DP03_0052E', 'DP03_0052PE', 'DP03_0053E', 'DP03_0053PE', 'DP03_0054E', 'DP03_0054PE', 'DP03_0055E', 'DP03_0055PE', 'DP03_0056E', 'DP03_0056PE', 'DP03_0057E', 'DP03_0057PE', 'DP03_0058E', 'DP03_0058PE', 'DP03_0059E', 'DP03_0059PE', 'DP03_0060E', 'DP03_0060PE', 'DP03_0061E', 'DP03_0061PE', 'NAME', 'state', 'county', 'FIPS']

DataFrame 'DF_th_w_earn':
['DP03_0064E', 'NAME', 'state', 'county', 'FIPS']

DataFrame 'DF_th_w_ss':
['DP03_0066E', 'DP03_0066PE', 'NAME', 'state', 'county', 'FIPS']

DataFrame 'DF_th_w_ri':
['DP03_0068E', 'DP03_0068PE', 'NAME', 'state', 'county', 'FIPS']

DataFrame 'DF_th_w_ssi':
['DP03_0070E', 'DP03_0070PE', 'NAME', 'state', 'county', 'FIPS']

DataFrame 'DF_th_w_cpa':
['DP03_0072E', 'DP03_0072PE', 'NAME', 'state', 'county', 'FIPS']

DataFrame 'DF_th_w_snap':
['DP03_0074E', 'DP03_0074PE', 'NAME', 'state', 'county', 'FIPS']

DataFrame 'DF_families':
['DP03_0

### Changing Column Names

Changing names of dataframe columns based on dataframe width, with verification

In [22]:
col_names_five = ['Total','Location','State','County','FIPS']

col_names_six = ['Total','Percent','Location','State','County','FIPS']

col_names_twenty_four = ['Estimate Less than $10,000','Percent Less than $10,000',
                      'Estimate $10,000 to $14,999','Percent $10,000 to $14,999',
                      'Estimate $15,000 to $24,999','Percent $15,000 to $24,999',
                      'Estimate $25,000 to $34,999','Percent $25,000 to $34,999',
                      'Estimate $35,000 to $49,999','Percent $35,000 to $49,999',
                      'Estimate $50,000 to $74,999','Percent $50,000 to $74,999',
                      'Estimate $75,000 to $99,999','Percent $75,000 to $99,999',
                      'Estimate $100,000 to $149,999',
                      'Percent $100,000 to $149,999',
                      'Estimate $150,000 to $199,999',
                      'Percent $150,000 to $199,999','Estimate $200,000 or more',
                      'Percent $200,000 or more','Location','State','County',
                      'FIPS'
                      ]

# Create a dictionary mapping DataFrame width to the corresponding column names list
column_names_mapping = {
    5: col_names_five,
    6: col_names_six,
    24: col_names_twenty_four
}

# Iterate over each DataFrame in the dictionary
for name, df in dfs.items():
    # Get the width of the DataFrame
    width = len(df.columns)
    
    # Check if the width exists in the column_names_mapping dictionary
    if width in column_names_mapping:
        # Change the column names using the corresponding list from column_names_mapping
        new_column_names = column_names_mapping[width]
        df.columns = new_column_names

    # Print the name of the DataFrame and the updated column names
    print(f"DataFrame '{name}':")
    print(df.columns.tolist())
    print()  # Print an empty line for separation

DataFrame 'DF_total_households':
['Total', 'Location', 'State', 'County', 'FIPS']

DataFrame 'DF_th_range':
['Estimate Less than $10,000', 'Percent Less than $10,000', 'Estimate $10,000 to $14,999', 'Percent $10,000 to $14,999', 'Estimate $15,000 to $24,999', 'Percent $15,000 to $24,999', 'Estimate $25,000 to $34,999', 'Percent $25,000 to $34,999', 'Estimate $35,000 to $49,999', 'Percent $35,000 to $49,999', 'Estimate $50,000 to $74,999', 'Percent $50,000 to $74,999', 'Estimate $75,000 to $99,999', 'Percent $75,000 to $99,999', 'Estimate $100,000 to $149,999', 'Percent $100,000 to $149,999', 'Estimate $150,000 to $199,999', 'Percent $150,000 to $199,999', 'Estimate $200,000 or more', 'Percent $200,000 or more', 'Location', 'State', 'County', 'FIPS']

DataFrame 'DF_th_w_earn':
['Total', 'Location', 'State', 'County', 'FIPS']

DataFrame 'DF_th_w_ss':
['Total', 'Percent', 'Location', 'State', 'County', 'FIPS']

DataFrame 'DF_th_w_ri':
['Total', 'Percent', 'Location', 'State', 'County', 'F

### Final verification before export

verifying count, name, and shape before export.<br>
Should match planning doc:<br>
'2020_income_benefit.xlsx'

In [23]:
# Get the count of dataframes in the dictionary
num_dataframes = len(dfs)

# Iterate over each DataFrame in the dictionary
for index, (name, df) in enumerate(dfs.items(), start=1):
    # Print the count, name, and shape of the DataFrame
    print(f"DataFrame {index}/{num_dataframes} - '{name}':")
    print(f"Count: {len(df)}")
    print(f"Shape: {df.shape}")
    print()  # Print an empty line for separation

DataFrame 1/15 - 'DF_total_households':
Count: 3221
Shape: (3221, 5)

DataFrame 2/15 - 'DF_th_range':
Count: 3221
Shape: (3221, 24)

DataFrame 3/15 - 'DF_th_w_earn':
Count: 3221
Shape: (3221, 5)

DataFrame 4/15 - 'DF_th_w_ss':
Count: 3221
Shape: (3221, 6)

DataFrame 5/15 - 'DF_th_w_ri':
Count: 3221
Shape: (3221, 6)

DataFrame 6/15 - 'DF_th_w_ssi':
Count: 3221
Shape: (3221, 6)

DataFrame 7/15 - 'DF_th_w_cpa':
Count: 3221
Shape: (3221, 6)

DataFrame 8/15 - 'DF_th_w_snap':
Count: 3221
Shape: (3221, 6)

DataFrame 9/15 - 'DF_families':
Count: 3221
Shape: (3221, 5)

DataFrame 10/15 - 'DF_fm_range':
Count: 3221
Shape: (3221, 24)

DataFrame 11/15 - 'DF_per_capita':
Count: 3221
Shape: (3221, 5)

DataFrame 12/15 - 'DF_non_family':
Count: 3221
Shape: (3221, 5)

DataFrame 13/15 - 'DF_worker_earn':
Count: 3221
Shape: (3221, 5)

DataFrame 14/15 - 'DF_worker_ft_earn_male':
Count: 3221
Shape: (3221, 5)

DataFrame 15/15 - 'DF_worker_ft_earn_female':
Count: 3221
Shape: (3221, 5)



### Export to excel
Create workbook and export wanted dataframes to excel as individual sheets.

In [24]:
# Define the file name
file_name = '2020_income_benefit_data.xlsx'

# Get the file path in the current working directory
file_path = os.path.join(os.getcwd(), file_name)

# Check if the file exists
if os.path.exists(file_path):
    # Load the existing Excel file
    excel_file = pd.read_excel(file_path, engine='openpyxl')
    
    # Create a new ExcelWriter object using the existing file
    writer = pd.ExcelWriter(file_path, engine='openpyxl', if_sheet_exists = 'replace', mode= 'a')
    
    # Iterate through the dataframes in dfs
    for df_name, df in dfs.items():
        # Check if the sheet already exists in the Excel file
        if df_name in file_path:
            # Remove the existing sheet
            writer.book.remove(writer.book[df_name])
        
        # Write the dataframe to a separate sheet in the Excel file
        df.to_excel(writer, sheet_name=df_name, index=False)
    
    # Save the Excel file
    writer.close()
else:
    # Create a new workbook
    writer = pd.ExcelWriter(file_path, engine='openpyxl')
    
    # Iterate through the dataframes in dfs
    for df_name, df in dfs.items():
        # Write each dataframe to a separate sheet in the Excel file
        df.to_excel(writer, sheet_name=df_name, index=False)
    
    # Save the Excel file
    writer.close()